<a href="https://colab.research.google.com/github/Aishani17/intel-project_1/blob/main/intel_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Path to your models
#/content/drive/MyDrive/TeacherModelalz/teacher_unet_alz.h5
teacher_model_path = "/content/drive/MyDrive/TeacherModelalz/teacher_unet_alz.h5"
student_model_path = "/content/drive/MyDrive/StudentModel3/stud_model_kd3.h5"


Mounted at /content/drive


In [2]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [3]:
!ngrok config add-authtoken 2zEppHKpYo5Nge9QJVeXMxAaH84_7XcsfBjVMWqrGAMVpBefZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import io

# --- Page Configuration ---
st.set_page_config(page_title="KD Demo: Teacher vs Student", layout="centered")

# --- Custom CSS Styling ---
st.markdown("""
    <style>
        .main {
            background-color: #f7f9fc;
        }
        .title {
            font-size: 32px;
            font-weight: bold;
            color: #4a90e2;
            text-align: center;
        }
        .subheader {
            color: #555;
            font-size: 20px;
            text-align: center;
            margin-top: -10px;
        }
        .footer {
            font-size: 14px;
            text-align: center;
            color: #999;
            margin-top: 30px;
        }
        .uploaded-image {
            border-radius: 10px;
            margin-bottom: 15px;
        }
    </style>
""", unsafe_allow_html=True)

# --- Title and Subtitle ---
st.markdown('<div class="title">Knowledge Distillation </div>', unsafe_allow_html=True)
st.markdown('<div class="subheader">Compare Outputs of Teacher and Student Models on Your Uploaded Image</div>', unsafe_allow_html=True)

# --- Load Models ---
@st.cache_resource
def load_models():
    teacher = load_model("/content/drive/MyDrive/TeacherModelalz/teacher_unet_alz.h5", compile=False)
    student = load_model("/content/drive/MyDrive/StudentModel3/stud_model_kd3.h5", compile=False)
    return teacher, student

# --- Image Pre/Post Processing ---
def preprocess_image(image):
    image = image.resize((128, 128))  # Resize as per model input
    image_array = np.array(image) / 255.0
    return np.expand_dims(image_array, axis=0)

def postprocess_image(pred):
    pred = np.clip(pred[0], 0, 1) * 255
    return Image.fromarray(pred.astype('uint8'))

# --- Load Models ---
teacher_model, student_model = load_models()

# --- Upload Interface ---
uploaded_file = st.file_uploader("📁 Upload an image (JPG/PNG)", type=["jpg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="🖼️ Original Image", use_column_width=True, output_format="PNG")

    input_data = preprocess_image(image)

    col1, col2 = st.columns(2)

    with col1:
        st.subheader(" Teacher Model Output")
        teacher_output = teacher_model.predict(input_data)
        teacher_result = postprocess_image(teacher_output)
        st.image(teacher_result, use_column_width=True)

    with col2:
        st.subheader(" Student Model Output")
        student_output = student_model.predict(input_data)
        student_result = postprocess_image(student_output)
        st.image(student_result, use_column_width=True)

# --- Footer ---
st.markdown('<div class="footer">Made with  using Streamlit</div>', unsafe_allow_html=True)


Writing app.py


In [5]:
!streamlit run app.py &>/content/logs.txt &


In [7]:
from pyngrok import ngrok

public_url = ngrok.connect("http://localhost:8501")
print("🔗 Your Streamlit app is live at:", public_url)


🔗 Your Streamlit app is live at: NgrokTunnel: "https://33de81807c97.ngrok-free.app" -> "http://localhost:8501"


In [ ]:

ngrok.kill()
